# Use Case 3: User interface to research intra-anaesthesisa hypotension
<span style = "font-size:22px">This notebook illustrates the usage of the vitabel package to visualize, annotate and process time-series data from the medical field. Please find the detailed, searchable documentation here: 
[![Documentation Status](https://readthedocs.org/projects/vitabel/badge/?version=latest)](https://vitabel.readthedocs.io/en/latest/?badge=latest)<br>
In this case we analyze non-waveform data from an anesthesia chart and add further labels to this data. This notebook in particular displays how `vitabel` can be used outside of resuscitation science and the plotting can be wrapped into overarching user interfaces. </span>

In [1]:
from vitabel import Vitals, Label

## 1) Load data

<span style = "font-size:18px">A vitabel object is initialized and data which was saved with vitabel before is loaded again.

In [2]:
case = Vitals()
case.load_data("data/usecase_3.json")

TypeError: IntervalLabel.__init__() got an unexpected keyword argument 'vline_text_source'

<span style = "font-size:18px">We get an overview over all channels and labels in the signal

In [ ]:
case.info()

## 3) Interactively plot and label data

<span style = "font-size:18px">A new label for text is initialized and added to the vitabel object.

In [ ]:
AnesthesiaLabel = Label(
    name = "Anesthesia", 
    time_index = [], 
    data = [], 
    plotstyle = {"linestyle": "--", "marker": None, "color": "teal"}
)
case.add_global_label(AnesthesiaLabel)

<span style = "font-size:18px">An interactive plot is initialized. Then the appearence of the figure is adapted, to better suit the use case. It is shown in the end.

In [ ]:
plot = case.plot_interactive(
    channels = [[0, 1, 2], [], []],
    labels = [["Event", "Anesthesia"], ["Remifentanil", "Medication"], ["Sevofluran"]],
    subplots_kwargs = {"figsize": (21, 9), "gridspec_kw": {"height_ratios": [5, 1, 0.5]}},
)
fig = plot.center.figure
ax = fig.get_axes()
ax[0].set_ylabel("Blood pressure (mmHg)")
ax[1].grid(False)
ax[2].grid(False)
fig.subplots_adjust(hspace = 0.03)
plot

## 4) Save data

<span style = "font-size:18px">The data is saved again in a json file.

In [ ]:
case.save_data("data/usecase_3_final.json")

In [ ]:
def plot_spaga(case: Vitals, data_out_path : Path, tobe_reviewed: List[str], case_index: int, container :widgets.VBox, case_holder: Dict[str, Vitals]):   
    global axes, endpoints, widget_hbox

    padding_time=4 # minutes before and after plot 

    # to keep adaptions alive over redraws
    def on_draw(event, case=case):
        global axes, endpoints, widget_hbox

        def show_auc(case, ax, cutoff : int=65):
            MAP=case.get_label('MAP')
            if MAP.is_time_absolute():
                reference_time = MAP.time_start - plot_start
                time_index = MAP.time_index + reference_time
            time_index /= pd.to_timedelta(1, unit="h")
            y2=np.array([cutoff]*len(MAP))
            ax.fill_between(time_index,MAP.data,y2,where=(MAP.data <= cutoff),interpolate=True, facecolor=orange, alpha=.5)


        # check wether legend was redrawn
        if axes[0].get_legend():
            for ax in axes[:-1]:  
                ax.grid(False) #Remove grid
                ax.set_xlabel("") # Remove the x-axis label      
                leg = ax.get_legend()
                if leg:
                    leg.remove()  # Remove legend
                    
            axes[0].grid(axis='y', visible=True)  # optional: keep y-axis grid

            for ax in axes[1:-2]:
                ax.set_yticks([])  
            
            # Align labels to the left so they line up with the right axis line
            for label in axes[2].get_yticklabels():
                label.set_horizontalalignment('right')

        # check if AUC is marked
        has_fill_between = any(isinstance(col, PolyCollection) for col in axes[0].collections)
        if not has_fill_between:
            show_auc(case, axes[0], cutoff=65)

        # calculate edpoints
        endpoints = calculate_endpoints(case)

        # Display Results
        widget_hbox["twa"].value = endpoints["twa"]
        widget_hbox["auc"].value = endpoints["auc"]
        widget_hbox["hypotens_dur"].value = endpoints["hypotens_dur"]
        widget_hbox["anae_dur"].value = endpoints["anae_dur"]
        widget_hbox["noa_equivalent"].value = endpoints["noa_equivalent"]
        widget_hbox["min_map"].value = endpoints["min_map"]
        widget_hbox["max_period_length_no_map"].value = endpoints["max_period_length_no_map"]
        widget_hbox["mean_period_length"].value = endpoints["mean_period_length"]

    t_analysis= case.get_label("Analysis").get_data()[0]
    plot_start=t_analysis.min() - pd.to_timedelta(padding_time, "m")
    plot_stop=t_analysis.max() + pd.to_timedelta(padding_time, "m")

    # Actual plotting
    plot= case.plot_interactive(channels=[["sys", "dia","map"],
                                          [],
                                          []],
                                labels=[['Anästhesie','Narkose','Chirurgie',"MAP","_dummy","Analysis",], 
                                        ['_localanaesthetics_bolus','_opioids_bolus', '_narcotics_bolus', '_relaxans_bolus', '_pressors_bolus', "_dummy", "Analysis"],
                                        ["_narcotics_cont", "_pressors_cont","_opioids_cont", "_dummy", "Analysis"]],     
                                channel_overviews=[["map"]],
                                subplots_kwargs={"figsize": (10, 5), "gridspec_kw": {"height_ratios": [8, 3, 2,1]}},
                                start=plot_start,
                                stop=plot_stop,
                                time_unit="h")
    fig = plot.center.figure
    axes = fig.get_axes()

    fig.canvas.mpl_connect('draw_event', lambda event: on_draw(event, case))
    fig.subplots_adjust(hspace=0)
    
    # Adapt plot
    axes[0].set_ylabel("Blood pressure (mmHg)")
    axes[0].xaxis.set_ticks_position('top')
    ymin,ymax=axes[0].get_ylim()
    axes[0].set_ylim(0,ymax)
    axes[3].set_ylim(0,ymax)

    text_labels = axes[0].findobj(lambda artist: isinstance(artist, Text) and hasattr(artist, "_from_vitals_label"))
    for artist in text_labels:
        artist.set_y(ymin + 0.1 * (ymax - ymin))

    axes[1].set_yticks([])  # Remove the y-axis label
    axes[1].set_ylim(0,1) # force to redraw subplot 1

    for ax in axes[1:-1]:
        ax.set_xticks([])  
    
    ymin,ymax=axes[2].get_ylim()
    ylength = ymax - ymin
    axes[2].set_ylim(ymin-ylength/4, ymax+ylength/4)
    
    # Move ticks and labels to the right side
    axes[2].yaxis.tick_right()
    axes[2].yaxis.set_label_position("right")
    # Move y-tick labels inside the plot
    axes[2].tick_params(axis='y', direction='in', pad=-5)  # Adjust pad to move inside

    return _wrap_in_widget(case=case, plot=plot, data_out_path=data_out_path, tobe_reviewed=tobe_reviewed, case_index=case_index, container=container, case_holder=case_holder)


